# **このKaggleコンペについて** 　　　「リクルートのレストランの訪問者を予測」

###   ///客の予約時のデータと実際に来店した際のデータを使用して、将来の各レストランに訪れる客の数を予測するというもの\\\

### air: Airレジ（飲食店向けPOSレジアプリ）の情報　/ 　hpg: ホットペッパーの情報

In [ ]:
# 与えられたinput用データを確認
# -----------------------------------
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# 基本モジュールをインポートする
# -----------------------------------
import pandas as pd    # data processing(情報処理する)
import numpy as np    # linear algebra(線形の形にする)
import scipy as sp    # Science and Technology Calculation(科学技術計算する)
import matplotlib.pyplot as plt    # 可視化のためのモジュール
%matplotlib inline
import seaborn as sns    # 可視化のためのモジュール
import warnings    # 不要な警告を無視する(出さない)ためのモジュール
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import Image    # 画像表示のためのモジュール

# # 分析のためのモジュールをインポート(SKLEARNはMLのためのモジュール)
# -----------------------------------
from sklearn.preprocessing import StandardScaler    # データの標準化
from sklearn.linear_model import LogisticRegression    # ロジスティック回帰
from sklearn import linear_model    # 回帰
from sklearn.tree import DecisionTreeClassifier    # 決定木
from sklearn.metrics import accuracy_score    # 精度評価
from sklearn.metrics import classification_report,confusion_matrix    # モデルの良し悪しを判断する

# データセットのインポート
# -----------------------------------
air_reserve = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_reserve.csv.zip')
air_store = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_store_info.csv.zip')
air_visit = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_visit_data.csv.zip')
date_info = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/date_info.csv.zip')
hpg_reserve = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/hpg_reserve.csv.zip')
hpg_store = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/hpg_store_info.csv.zip')
sample = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/sample_submission.csv.zip')
store_id_relation = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/store_id_relation.csv.zip')

air_reserve.shape ,air_store.shape ,air_visit.shape ,date_info.shape ,hpg_reserve.shape ,hpg_store.shape ,sample.shape ,store_id_relation.shape

In [ ]:
air_reserve.head(3)

#来店者の情報(air)
#  Store id(air_store_id)
#  来店した日時(visit_datetime)
#  予約した日時(reserve_datetime)
#  来店者数(reserve_visitors)

In [ ]:
air_store.head(3)

#来店者の情報(air)
#  Store id(air_store_id)
#  店舗のジャンル(air_genre_name)
#  場所(air_area_name)
#  緯度 (latitude)
#  経度 (longitude)

In [ ]:
air_visit.head(3)

#来店者の情報(air)
#  Store id(air_store_id)
#  日付(visit_date)
#  来店者数(visitors)

In [ ]:
date_info.head(3)

#カレンダーの情報(data)
#  カレンダー(calendar_date)
#  曜日(day_of_week)
#  休日かどうか(holiday_flg) (1休日、祝日:,0:平日)

In [ ]:
hpg_reserve.head(3)

#予約の情報(hpg)
#  Store id(hpg_store_id)
#  来店した日時(visit_datetime)
#  予約した日時(reserve_datetime)
#  (行く予定の予約日時ではなく予約自体をした日時と思われる)
#  予約した人数(reserve_visitors)

In [ ]:
hpg_store.head(3)

#予約の情報(hpg)
#  Store id(hpg_store_id)
#  店舗のジャンル(air_genre_name)
#  場所(air_area_name)
#  緯度 (latitude)
#  経度 (longitude)

In [ ]:
sample.head(3)

#提出用ファイルの情報(sample)
#  Store id 日付(id)
#  来店者数(visitors)

In [ ]:
store_id_relation.head(3)

#全ストアid(air&hpg)
#  airレストランの来店者数(air_store_id)
#  ホットペッパーの来店者数(hpg_store_id)

In [ ]:
# 元々のIDからstore_idとvisit_dateを切り出す(sampleのid部分に時間とidが混在していて見にくいので分ける)

sample['store_id'] = sample['id'].str[:20]    # str[:20]は最初から20文字目までを取得している
sample['visit_date'] = sample['id'].str[21:]

# 日付型へ変換
sample['visit_date'] = pd.to_datetime(sample['visit_date'])

# store_idのカラム名をair_store_idに変える
sample.rename(index = str,columns={'store_id': 'air_store_id'})

# データの表示
sample.head(3)

In [ ]:
# 欠損のチェック

air_reserve.isnull().sum() , air_store.isnull().sum() , air_visit.isnull().sum() , date_info.isnull().sum() , hpg_reserve.isnull().sum() , hpg_store.isnull().sum() , store_id_relation.isnull().sum()

### 欠損値0

In [ ]:
# 来店者数と予約者数を可視化

f,ax = plt.subplots(1,1,figsize=(15,8))
plt1 = air_visit.groupby(['visit_date'], as_index=False).agg({'visitors': np.sum})
plt1=plt1.set_index('visit_date')
plt1.plot(color='salmon', kind='area', ax=ax)
plt.ylabel("Sum of Visitors")
plt.title("Visitor and Reservations")

## 上の図から2016-07から来店者数が上がったことが読み取れる

## なぜか？

# 要因予想：2016年の世界経済を見たときに、2016年前半は世界経済と国際金融市場が大きく動揺し,日本経済と東京市場も例外ではなく、円高・株安になっていた。
# 以上から、1年間のデータしか手元にはないが、毎年前半が来店者数が落ち込むわけではなく、突発的な出来事であり、2016年においては7月以降のデータが例年の来店者数であると推定できる

In [ ]:
# 来店者数と予約者数を24時間の時間で可視化

air_reserve['visit_hour'] = pd.to_datetime(air_reserve['visit_datetime']).dt.hour    # datetimeからdt.hourで時間だけ取り出して行列化する
air_reserve.groupby('visit_hour')['reserve_visitors'].sum().plot.bar(rot=0)    # pandasのメソッドの一つで、groupby()で指定した列でグルーピングする
plt.title("visit_hour & reserve_visitors")
plt.xlabel("visit_hour")
plt.ylabel("reserve_visitors")



## 下図から17:00-21:00の間に来店者数が多くなっていることが分かる

In [ ]:
# 店のジャンルを可視化

air_store.air_genre_name.value_counts().sort_values().plot(kind='barh')

## 先ほど述べた、世界経済の落ち込みによる影響を受けやすい居酒屋やバーが上位に来ている

In [ ]:
# 予約システムの偏り調査

air = pd.merge(air_reserve,air_store,on='air_store_id')    # pd.merge()関数では第一引数(air_reserve)と第二引数(air_store)に2つを結合するpandas.DataFrame
hpg = pd.merge(hpg_reserve,hpg_store,on='hpg_store_id')    # pandas.DataFrameを縦横に連結するにはpandas.concat()関数
airrel = pd.merge(air,store_id_relation,how='left',on='air_store_id')    # leftなので左から結合
hpgrel = pd.merge(hpg,store_id_relation,how='left',on='hpg_store_id')
full = pd.merge(airrel,hpgrel,how='outer')
print("airのレストランが ",len(air)," 店舗、 ホットペッパーのレストランが ",len(hpg)," 店舗で、そのうち",len(store_id_relation),' 店舗が両方使っている。')

## 予約システムはホットペッパーに偏りがあると得られた

In [ ]:
# 店のジャンルををダミー変数化する
air_store = pd.concat([air_store, pd.get_dummies(air_store['air_genre_name'], prefix = 'air_genre_name')], axis=1)
air_store.head(3)

In [ ]:
# 休日

date_info.loc[date_info.holiday_flg==1]

In [ ]:
train = pd.merge(air_reserve,store_id_relation,how="inner",on=["air_store_id"])
train = pd.merge(train,air_store,how="inner",on=["air_store_id"])
train = pd.merge(train,air_visit,how="inner",on=["air_store_id"])
train1 = train.drop(["visit_date","visit_datetime","hpg_store_id","reserve_datetime", "air_genre_name", "air_area_name", "latitude", "longitude","visitors"], axis = 1)
train = train.sample(1000)
train1 = train1.sample(1000)
train

In [ ]:
# 説明変数と目的変数を指定

# 説明変数のデータを作る
X = train.drop(["air_store_id","visit_date","visit_datetime","hpg_store_id","reserve_datetime", "air_genre_name", "air_area_name", "latitude", "longitude","visitors"], axis = 1)

# 目的変数のデータを作る
y = train['visitors'].values 

In [ ]:
# 重回帰のモデリング
# モデルのインスタンスを作る
reg2 = linear_model.LinearRegression()
# データをインプットする
reg2.fit(X,y) # この時点でidとつなげないとできない(air_visit✖列行違うため)

In [ ]:
print('回帰係数は',reg2.coef_,'\n') #アンダーバー(_)に注意、忘れないように！
print('切片は',reg2.intercept_)

In [ ]:
X2 = pd.merge(sample,train1,how="inner",left_on='store_id', right_on='air_store_id')
X3 = X2.drop(["air_store_id","id","store_id"], axis = 1)
X3

In [ ]:
# モデルを使った予測値を出力
reg2.predict(X3)

In [ ]:
sample.visitors = reg2.predict(X2)

In [ ]:
sample

In [ ]:
sample.to_csv("sample.csv")

In [ ]:

sample.drop(['visitors'], axis=1, inplace=True)    # axisは次元を指定しており、axis=0は行、axis=1は列を表す（デフォルトは0）# inplace=Trueとすると、元のデータフレームが変更される。


In [ ]:
vdt = pd.to_datetime(full.visit_datetime)
rdt = pd.to_datetime(full.reserve_datetime)
full['vd']=vdt.dt.date
full['vt']=vdt.dt.time
full['rd']=rdt.dt.date
full['rt']=rdt.dt.time

dts = pd.to_datetime(date_info.calendar_date)
days = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
date_info['calendar_date'] = pd.to_datetime(date_info['calendar_date']).dt.date
date_info['dy'] = dts.dt.dayofyear
date_info['dw'] = [days.index(dw) for dw in date_info.day_of_week]
date_info.head()

nf = pd.merge(full,date_info[['calendar_date','holiday_flg']],how='left',left_on='vd',right_on='calendar_date')
nf = nf.rename(index = str, columns = {'holiday_flg':'visit_holiday'})
nf = nf.drop(['calendar_date'],axis=1)

nf = pd.merge(nf,date_info[['calendar_date','holiday_flg']],how = 'left', left_on='rd',right_on='calendar_date')
nf = nf.rename(index = str, columns = {'holiday_flg':'reservation_holiday'})
nf = nf.drop(['calendar_date'],axis=1)

nf['vd'] = pd.to_datetime(nf['vd']).dt.dayofyear
nf['rd'] = pd.to_datetime(nf['rd']).dt.dayofyear
nf

In [ ]:
X1 = sample.loc[:, ["visit_date"]].values
X1